In [2]:
import sys
sys.path.append("..")
from src.utilities import *

result = transcribe("../artifacts/audio/example.mp3", "medium.en")

  1%|          | 2756/320960 [00:36<1:09:31, 76.28frames/s]


KeyboardInterrupt: 

### Chapterization
Feed segments to Llama-70b to generate chapter titles.

In [3]:
import json
import sys
sys.path.append("..")
from src.utilities import *


# Segment the transcript on all pauses
# segments = chunk_on_pause(result['segments'], -1, 250, 1500)

with open('tmp.txt', 'r') as file:
    segments = json.loads(file.read())

# Create Chapter titles for segments
chapters = chapterize(segments[:5])

for c in chapters:
    print(c["title"])
    print(c["time"])
    print(c["metadata"], "\n")
    print(c["text"], "\n\n")

with open('../artifacts/data/chapters.json', 'w') as file:
    file.write(json.dumps(chapters, indent=4))

"Introduction to MIT's Course 600: Computer Science and Programming"
[00:00:00,000 --> 00:06:07,000]
(Chunk 1, 1167 words, 0:06:07.120000 mins) 

 The following content is provided under a Creative Commons license. Your support will help MIT OpenCourseWare continue to offer high quality educational resources for free. To make a donation or view additional materials from hundreds of MIT courses, visit MIT OpenCourseWare at ocw.mit.edu. PROFESSOR ERIC GRIMSON Good morning. Try it again. Good morning. Thank you. This is 600, also known as Introduction to Computer Science and Programming. My name is Eric Grimson. Together, Professor John Guttag over here. We're going to be lecturing the course this term. I want to give you a heads up. You're getting some serious firepower this term. John was department head for 10 years, felt like a century. And in course 6, I'm the current department head in course 6. John's been lecturing for 30 years, roughly. All right. I'm the young guy. I've only bee

### Note Generation
Feed the chapters back to Llama-70b to create useful notes from the transcript.

In [16]:
import json
import sys
sys.path.append("..")
from src.utilities import *

# q = "Imagine you are attending the lecture and tasked with refining the following excerpt into notes. Your goal is to streamline the content by eliminating redundancies and extraneous details. As you organize the notes, feel free to create subheadings where necessary. Your focus should be on retaining crucial information.\n\n"
q = "Write structured notes ONLY in bulletpoint form using the following excerpt of a lecture transcript:\n"

with open('../artifacts/data/chapters.json', 'r') as file:
    chapters = json.loads(file.read())

note_arr=[]
for c in chapters:
    content = '\n'.join([c['title'], c['metadata'], c['text']])
    
    notes = query(q, content)
    filtered ='\n'.join([line for line in notes.split('\n') if line.strip() and line.strip()[0] in ('*', '+', '-')])
    
    print('\n', filtered, '\n')
    
    note_arr.append({'title': c['title'], 'content': filtered})
    
with open('../artifacts/data/notes.json', 'w') as file:
    file.write(json.dumps(note_arr, indent=4))


 - Lecture on Introduction to MIT's Course 600: Computer Science and Programming
- Professors Eric Grimson and John Guttag are teaching the course
- Course is taken seriously and aims to help everyone learn about computation
- Three main topics for lecture: administrivia, goals of the course, and concepts and tools of computational thinking
- Strategic goals of the course: prepare freshmen and sophomores for majoring in Course 6, help non-majoring students feel confident in coding, provide understanding of the role of computation in technical problems, and prepare students for jobs and internships
- Course is for students with little or no prior programming experience
- Overqualified students should talk to the professors for alternatives
- Tactical goals of the course: acquire skills in computational thinking, understand and write small scale programs, use vocabulary of computational tools, read and analyze programs written by others, understand the capabilities and limitations of co

In [17]:
# Make notes of the notes

q = "Write a 1 sentence <50 characters summary of the notes provided. Please return ONLY the summary.\n"

sum_arr = []
for n in note_arr:
    summary = query(q, n['content'])
    print(summary)
    sum_arr.append({'title': n['title'], 'content': summary})
    
with open('../artifacts/data/summary.json', 'w') as file:
    file.write(json.dumps(sum_arr, indent=4))

Introduction to MIT's Course 600: Computer Science and Programming; taught by Professors Eric Grimson and John Guttag; aims to help everyone learn about computation; strategic and tactical goals include preparing students for majoring in Course 6, building coding confidence, understanding computation's role in technical problems, and acquiring various computational skills.
Learn to map scientific problems into a computational framework through lectures, recitations, and problem sets in Python, with an emphasis on understanding the limitations and capabilities of computation.
Two quizzes scheduled (Oct 2nd, Nov 4th), check website for conflicts; problem sets help learn; seek help if needed.
Recitation signup, optional textbook, encouraged attendance, extensive material covered, TAs may lack patience, responsible behavior expected, no handed out class notes, valuable note-taking.
Handouts provided with code examples for lectures; notes may not be clear without attending lecture; personal

In [18]:
import json
import sys
sys.path.append("..")
from src.utilities import *

# Make summary of the summary
p= "Write a 3 sentence abstract of the following notes.\n"

with open('../artifacts/data/summary.json', 'r') as file:
    sum_arr = json.loads(file.read())

txt = '\n'.join([s['content'] for s in sum_arr])

abstract = query(p, txt)

print(abstract)

with open('../artifacts/data/abstract.txt', 'w') as file:
    file.write(abstract)

The introduction to MIT's Course 600 in Computer Science and Programming, taught by Professors Eric Grimson and John Guttag, has the goal of helping individuals learn about computation. The course aims to prepare students for majoring in Course 6, build coding confidence, understand computation's role in technical problems, and acquire various computational skills. The curriculum includes lectures, recitations, and problem sets in Python, with a focus on understanding the limitations and capabilities of computation. Two quizzes are scheduled, and students should check the website for any conflicts. Problem sets are designed to aid in learning, and students are encouraged to seek help if needed. Recitation sign-up is required, an optional textbook is suggested, and attendance is strongly encouraged. Extensive material is covered, and TAs may lack patience, so responsible behavior is expected. Class notes are not handed out, so valuable note-taking is recommended. Handouts with code exam

In [19]:
import json
import sys
sys.path.append("..")
from src.utilities import *

# Create a md file with everything we have

with open("../artifacts/data/abstract.txt", 'r') as f:
    abstract = f.read()

with open("../artifacts/data/summary.json", 'r') as f:
    summary = json.loads(f.read())

with open("../artifacts/data/notes.json", 'r') as f:
    notes = json.loads(f.read())
    
# Generate the markdown file
sections=[]

for s, n in zip(summary, notes):
    title = n['title']
    n_body = n['content']
    summary_body = s['content']
    
    # Format note_body into markdown
    p="Format the following notes into rich markdown.\n Indent knowledge, bold keywords, and italicize examples. Format appropriate data into tables. Last but not least, keep the headers under or equal to ###.\n"
    
    formatted = query(p, n_body)
    
    section_md = f"## {title}\n{summary_body}\n{formatted}\n"
    
    sections.append(section_md)
    print(section_md)

md = f"""# Lecture Notes

## Abstract
{abstract}

{''.join(sections)}
"""

with open("../artifacts/data/notes.md", 'w') as f:
    f.write(md)

## "Introduction to MIT's Course 600: Computer Science and Programming"
Introduction to MIT's Course 600: Computer Science and Programming; taught by Professors Eric Grimson and John Guttag; aims to help everyone learn about computation; strategic and tactical goals include preparing students for majoring in Course 6, building coding confidence, understanding computation's role in technical problems, and acquiring various computational skills.
### Lecture on Introduction to MIT's Course 600: Computer Science and Programming

- Professors **Eric Grimson** and **John Guttag** are teaching the course.
- The course is taken seriously and aims to help **everyone learn about computation**.
- Three main topics for the lecture are:
    - administrivia
    - goals of the course
    - concepts and tools of computational thinking
    
### Strategic Goals of the Course

- Prepare freshmen and sophomores for majoring in **Course 6**.
- Help non-majoring students feel confident in coding.
- Provide 

In [7]:
# Find difficult terms

import json
import sys
sys.path.append("..")
from src.utilities import *

with open('../artifacts/data/notes.json', 'r') as file:
    notes = json.loads(file.read())
    
q = "Identify the domain-specific jargon in the following notes, and return them in a SEMICOLON SEPARATED FORMAT.\n"

terms = set()
for n in notes:
    print(n['title'])
    
    o = query(q, n['content'])
    print(o)
    
    # Find difficult terms
    t = [t.strip() for t in o.split(';')]
    terms.update(t)
    
with open('../artifacts/data/terms.txt', 'w') as file:
    file.write(';'.join(terms))
    

"Introduction to MIT's Course 600: Computer Science and Programming"
domain-specific jargon: 
- Creative Commons license
- Introduction to Computer Science and Programming
- Course 600
- lecturers
- computation
- computational thinking
- computer scientist
- Course 6
- programming
- code
- technical problems
- confidence and competence
- small-scale programs
- features, limits, and costs of computation
- limitations of computation
"Mapping Scientific Problems into a Computational Framework: An Introductory Course"
tactical skill; course; computational framework; problem description; computational language; computation; tools; course administration details; lecture; class timing; recitation; class work; problem sets; programming; Python; difficulty level; term; libraries; code
"Course Overview and Expectations"
course materials; course codes; quizzes; conflicting schedule; special needs; open book/notes; grading distribution; problem sets; final exam; challenge level; platform; paperles

In [8]:
# Generate definitions for difficult terms

import json
import sys
sys.path.append("..")
from src.utilities import *

with open('../artifacts/data/terms.txt', 'r') as file:
    terms = file.read().split(';')

q = "Provide a definition for the following term.\n"

defs = {}
for term in terms:
    print(term)
    definition = query(q, term)
    print(definition)
    
    defs[term] = definition

with open('../artifacts/data/definitions.json', 'w') as file:
    file.write(json.dumps(defs, indent=4))

OpenCourseWare
OpenCourseWare refers to free and open-access educational resources, including course materials, lectures, assignments, and other educational content that is made available online to the public. OpenCourseWare allows individuals to access and utilize educational materials from universities, colleges, and other educational institutions without having to enroll in formal programs or pay tuition fees. It aims to promote the sharing of knowledge, increase access to education, and foster lifelong learning for people around the world. OpenCourseWare typically includes materials from a wide range of disciplines, allowing individuals to learn at their own pace and explore various subjects of interest.
